### mysql과 python 연동
1. 라이브러리 설치 
2. sql 서버의 정보를 등록
3. sql 서버에 접속 
4. sql 쿼리문을 이용하여 데이터를 CRUD작업

In [ ]:
# 라이브러리 설치 
!pip install pymysql

In [ ]:
!pip install pandas
!pip install matplotlib

In [ ]:
# 라이브러리 로드 
import pymysql

In [ ]:
import pandas as pd

In [ ]:
# sql 서버의 정보를 변수에 대입 
_db = pymysql.connect(
    user = 'root', 
    password = '1234', 
    # host = '172.16.133.45',
    host = 'localhost' ,
    db = 'ezen', 
    port = 3306
)

In [ ]:
# cursor 생성
# sql 쿼리문을 실행 시키기 위한 필수 변수
# Dictcursor는 sql 쿼리문을 실행하고 나서 결과값을 딕셔너리 형태로 받기 위한 설정
cursor = _db.cursor(pymysql.cursors.DictCursor)

In [ ]:
# sql 쿼리문을 변수로 저장
sql = """
    SELECT 
    * 
    FROM 
    user
"""

# sql 쿼리문을 서버에 질의
cursor.execute(sql)

In [ ]:
result = cursor.fetchall()
result

In [ ]:
pd.DataFrame(result)

In [ ]:
### insert 
sql = """
    INSERT INTO
    user 
    VALUES 
    ('test3', '111111', '3')
"""

cursor.execute(sql)

In [ ]:
_db.commit()

### sql 함수화
1. 첫번째 함수
    - 매개변수 1개(sql 쿼리문)
    - sql 쿼리문을 실행하고 결과를 데이터프레임으로 리턴
2. 두 번째 함수
    - 매개변수 1개(sql 쿼리문)
    - sql 쿼리문을 실행하고 데이터베이스를 변경 
3. 세 번째 함수
    - 1번 함수와 2번 함수를 하나의 함수에서 실행이 될 수 있도록 생성

In [ ]:
def sql_select(sql):
    # 커서를 이용하여 sql을 실행
    cursor.execute(sql)
    # 쿼리문에 대한 결과를 데이터프레임으로 변경
    result = cursor.fetchall()
    df = pd.DataFrame(result)
    return df

In [ ]:
sql = """
    SELECT 
    * 
    FROM 
    user
"""

sql_select(sql)

In [ ]:
def sql_cud(sql, value):
    # 커서를 이용하여 쿼리문을 실행
    cursor.execute(sql, value)
    # 데이터베이스를 변경 
    _db.commit()
    return "sql 변경 완료"

In [ ]:
sql = """
    INSERT INTO 
    user 
    VALUES 
    (%s, %s, %s)
"""

values = ["test5", "1234", "1"]

sql_cud(sql, values)

In [ ]:
sql = """
    insert into 
    user_select 
    values 
    (%s, %s)
"""

sql2 = "select * from user"

# 문자열을 찾는 함수
print(sql.find('select'))
print(sql2.find('select'))


In [ ]:
sql2.strip().upper().startswith('SELECT')

In [ ]:
def _sql(sql, value=[]):
    cursor.execute(sql, value)
    if sql.strip().upper().startswith('SELECT'):
        result = cursor.fetchall()
        df = pd.DataFrame(result)
        return df
    else:
        _db.commit()
        return "sql 변경 완료"

In [ ]:
sql = """
    select 
    * 
    from 
    user
"""

_sql(sql)

In [41]:
sql = """
    delete 
    from 
    user 
    where 
    id = %s
"""

values = ['test5']

_sql(sql, values)

'sql 변경 완료'

### sql 함수를 클래스로 변경 

In [42]:
class Database:
    # 생성자 함수 생성
    # DB 서버에 대한 정보를 입력
    def __init__(self, _host, _user, _pass, _db, _port):
        # db라는 변수 생성 (sql 서버에 접속을 하는 변수)
        self.db = pymysql.connect(
            user = _user, 
            password = _pass, 
            host = _host, 
            db = _db, 
            port = _port
        )
        # cursor 변수 생성
        self.cursor = self.db.cursor(pymysql.cursors.DictCursor)
    
    # 쿼리문을 실행하여 결과를 리턴해주거나 데이터베이스를 변경하는 함수
    def query(self, sql, value = []):
        self.cursor.execute(sql, value)
        
        if sql.strip().upper().startswith('SELECT'):
            result = self.cursor.fetchall() 
            result = pd.DataFrame(result)
        else:
            self.db.commit()
            result = "Query OK"
        
        return result
    
    # sql 서버와의 접속을 종료
    def sql_close(self):
        self.db.close()

In [43]:
# 클래스 생성
test_sql = Database(
    'localhost', 
    # '172.16.133.45'
    'root', 
    '1234', 
    'ezen', 
    3306
)

In [46]:
sql = """
    select 
    * 
    from 
    user
"""

test_sql.query(sql)

,id,password,no
0,asd,12345,
1,test,1234,
2,test2,9999,
3,test3,111111,3
4,test4,1111,10


In [45]:
_id = input("id를 입력하시오")
_pass = input('패스워드를 입력하시오')
sql = """
    update 
    user 
    set 
    password = %s
    where 
    id = %s
"""

value = [_pass, _id]

test_sql.query(sql, value)

'Query OK'

In [47]:
import sql_mod as sm

In [48]:
# 클래스 생성
mod_sql = sm.Database(
    # "172.16.133.45",
    'localhost', 
    # 'root2',
    'root', 
    '1234', 
    'ezen', 
    3306
)

In [49]:
sql = """
    select 
    * 
    from 
    user_info
"""

mod_sql.query(sql)

,id,name,adr,phone
0,test,kim,None,None
1,test2,park,Seoul,01012345678
2,asd,Lee,bundong,01078945623


In [50]:
mod_sql2 = sm.Database(
    '172.16.133.45', 
    "root2", 
    "1234", 
    "ezen", 
    3306
)

In [51]:
mod_sql2.query(sql)

,id,name,adr,phone
0,test,kim,None,None
1,test2,park,Seoul,01012345678
2,asd,Lee,bundong,01078945623


In [52]:
mod_sql.query(sql)

,id,name,adr,phone
0,test,kim,None,None
1,test2,park,Seoul,01012345678
2,asd,Lee,bundong,01078945623
